# Intermediate SQL Test Project

For this project we will use a SQLite database called chinook.db, which contains information about a fictional digital music shop.

The goal is to answer some business questions. For example, selecting 3 albums from a list of 4 to add to the store based on the information available in the database related to the artists that own those albums.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT
    *
FROM invoice
LIMIT 5;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


#### *"We were asked to select 3 albums from a group of 4 to add to the store. We only have information of the artist and the genre of those albums. Based on that we have to select the albums we will add to the store. We were asked to make this analysis based on USA sales."*

#### *"The 4 albums genres are: Hip-Hop, Punk, Pop, Blues."*

In [4]:
%%sql
WITH
    invoice_track AS
    (
    SELECT 
        inv.billing_country AS country,
        inv.invoice_id invoice_id,
        t.genre_id
    FROM track t
    INNER JOIN invoice_line inv_l ON t.track_id = inv_l.track_id
    INNER JOIN invoice inv ON inv_l.invoice_id = inv.invoice_id
    WHERE inv.billing_country = "USA"
    ),
    total_USA AS
    (
    SELECT 
        COUNT(*)
    FROM track t
    INNER JOIN invoice_line inv_l ON t.track_id = inv_l.track_id
    INNER JOIN invoice inv ON inv_l.invoice_id = inv.invoice_id
    WHERE inv.billing_country = "USA"
    )

    
SELECT 
    it.country country,
    g.name genre,
    COUNT(it.invoice_id) AS "Tracks per Genre",
    (SELECT * FROM total_USA) "Total Tracks sold USA",
    ROUND((CAST(COUNT(it.invoice_id) AS FLOAT) / CAST((SELECT * FROM total_USA) AS FLOAT)) * 100, 0) AS "Percentage Sold in USA per Genre"
FROM invoice_track it
INNER JOIN genre g ON it.genre_id = g.genre_id
GROUP BY genre
ORDER BY "Percentage Sold in USA per Genre" DESC;

Done.


country,genre,Tracks per Genre,Total Tracks sold USA,Percentage Sold in USA per Genre
USA,Rock,561,1051,53.0
USA,Alternative & Punk,130,1051,12.0
USA,Metal,124,1051,12.0
USA,R&B/Soul,53,1051,5.0
USA,Alternative,35,1051,3.0
USA,Blues,36,1051,3.0
USA,Hip Hop/Rap,20,1051,2.0
USA,Latin,22,1051,2.0
USA,Pop,22,1051,2.0
USA,Easy Listening,13,1051,1.0


Here are the top genres of music sold in USA.

Taking in cosideration the genres of the albums the store is trying to add, the order of priority should be: Punk, Blues, Hip-Hop, Pop.

The difference in sales between Hip-Hop and Pop is minimal, but still Hip-Hop have 2 more sales in USA.

#### *"We were asked to analyze the performance of the employees in sales."* 

#### *"The database contain information of the ammount of money spent by customer, and since each customer is assigned to a sales support employee, we can use that information to make a quick analysis."*

In [5]:
%%sql
WITH
    cust_inv AS
    (
    SELECT 
        cust.customer_id,
        cust.support_rep_id,
        SUM(inv.total) AS "total_sales_per_customer (dollars)"
    FROM customer cust
    INNER JOIN invoice inv ON cust.customer_id = inv.customer_id
    GROUP BY cust.customer_id
    )
    
SELECT 
    emp.first_name || " " || emp.last_name employee_name,
    emp.title employee_title,
    ROUND(SUM(ci."total_sales_per_customer (dollars)"), 2) AS "total_sales_per_employee (Dollars)"
FROM employee emp
LEFT JOIN cust_inv ci ON ci.support_rep_id = emp.employee_id
GROUP BY employee_name
ORDER BY employee_title;

Done.


employee_name,employee_title,total_sales_per_employee (Dollars)
Andrew Adams,General Manager,None
Michael Mitchell,IT Manager,None
Laura Callahan,IT Staff,None
Robert King,IT Staff,None
Nancy Edwards,Sales Manager,None
Jane Peacock,Sales Support Agent,1731.51
Margaret Park,Sales Support Agent,1584.0
Steve Johnson,Sales Support Agent,1393.92


We used the information from 3 tables of the database to obtain our analysis results.

When we checked the SUM of money spent by customers per each eamployee, we notice that only 3 employees have some kind of value, while the rest do not have sales regsitered.

We decided then to add the title of each employee, and then we noticed that the employees that do not have any sales ammounts, are all in non-sales positions, like management or IT.

Taking that into account and checking the ammount of sales of the 3 Sales Support Agents, we can see that some seems to have better performance that others, but do not seems to be a huge difference, waht make us think that the performance of those 3 employees is similar in this aspect.

In [6]:
%%sql
SELECT DISTINCT employee_id
FROM employee;

Done.


employee_id
1
2
6
3
4
5
7
8


Just in case we verify the ammount of employees and make sure none was left out of the analysis.

#### *"We were asked to analyze the sale data for customers from each country, and provide information like: total number of customers per country, total value of sales per country, average value of sales per customer from each country, and average order value."*

In [7]:
%%sql
WITH cust_country AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 WHERE country = cust.country
                ) <= 1 THEN "Other"
           ELSE cust.country
       END AS country,
       cust.customer_id customer_id,
       inv_l.unit_price unit_price,
       inv_l.quantity quantity,
       inv_l.invoice_id
     FROM invoice_line inv_l
     INNER JOIN invoice inv ON inv.invoice_id = inv_l.invoice_id
     INNER JOIN customer cust ON cust.customer_id = inv.customer_id
    )
    
SELECT  
    cc.country,
    COUNT(DISTINCT cc.customer_id) AS "Total Number of Customers",
    ROUND(SUM(cc.unit_price), 2) AS "Total Value of Sales",
    ROUND(SUM(cc.unit_price) / COUNT(DISTINCT cc.customer_id), 2) AS "Average Value of Sales per Customer",
    ROUND(SUM(cc.unit_price) / COUNT(DISTINCT cc.invoice_id), 2) AS "Average Order Value"
FROM (
     SELECT
        *,
        CASE
            WHEN cc.country = "Other" THEN 1
            ELSE 0
        END AS sort
     FROM cust_country cc
    ) cc
GROUP BY cc.country
ORDER BY sort ASC, "Total Value of Sales" DESC;

Done.


country,Total Number of Customers,Total Value of Sales,Average Value of Sales per Customer,Average Order Value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


Using 3 different tables, we were able to gather the information requested.

As we can see, USA is the country with the highest ammount of spent money in the store, and that is most likely related to the higher ammount of customers from that country.

#### *"The shop want to know if the customers tend to buy music track individually or a whole set of an album."*

#### *"This information would help the shop to know if is better to buy from record companies only the most famous tracks of each album, or buy the whole album."*

In [8]:
%%sql
CREATE VIEW alb_inv_track AS
    SELECT 
        inv_l.invoice_id,
        t.album_id,
        t.track_id
    FROM track t
    INNER JOIN invoice_line inv_l ON t.track_id = inv_l.track_id
    INNER JOIN invoice inv ON inv.invoice_id = inv_l.invoice_id
    WHERE 
        (
        SELECT COUNT(*)
        FROM track
        WHERE album_id = t.album_id
        ) > 3
    GROUP BY inv_l.invoice_id
    ORDER BY inv_l.invoice_id;

(sqlite3.OperationalError) table alb_inv_track already exists
[SQL: CREATE VIEW alb_inv_track AS
    SELECT 
        inv_l.invoice_id,
        t.album_id,
        t.track_id
    FROM track t
    INNER JOIN invoice_line inv_l ON t.track_id = inv_l.track_id
    INNER JOIN invoice inv ON inv.invoice_id = inv_l.invoice_id
    WHERE 
        (
        SELECT COUNT(*)
        FROM track
        WHERE album_id = t.album_id
        ) > 3
    GROUP BY inv_l.invoice_id
    ORDER BY inv_l.invoice_id;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


Here we created a view that we will use more than once. In this view we selected only the albums that have more than 3 tracks.

In [9]:
%%sql
SELECT *
FROM alb_inv_track;

Done.


invoice_id,album_id,track_id
1,91,1173
2,215,2651
3,214,2646
4,203,2514
5,163,2002
6,5,30
7,6,42
8,125,1560
9,20,196
10,257,3290


In [10]:
%%sql

SELECT 
    ait.invoice_id,
    ait.album_id,
    CASE
        WHEN
            (
                (
                SELECT inv_l.track_id FROM invoice_line inv_l
                WHERE inv_l.invoice_id = ait.invoice_id
            
                EXCEPT
                
                SELECT t.track_id FROM track t
                WHERE t.album_id = ait.album_id
                )
                
                    AND
                
                (
                SELECT t.track_id FROM track t
                WHERE t.album_id = ait.album_id
                EXCEPT
                
                SELECT inv_l.track_id FROM invoice_line inv_l
                WHERE inv_l.invoice_id = ait.invoice_id
                )    
            ) IS NULL THEN "Album Purchase"
        ELSE "Not Album Purchase"
    END Type_of_purchase
FROM alb_inv_track ait
GROUP BY ait.invoice_id
ORDER BY ait.invoice_id
;

Done.


invoice_id,album_id,Type_of_purchase
1,91,Album Purchase
2,215,Not Album Purchase
3,214,Not Album Purchase
4,203,Not Album Purchase
5,163,Album Purchase
6,5,Not Album Purchase
7,6,Not Album Purchase
8,125,Not Album Purchase
9,20,Not Album Purchase
10,257,Not Album Purchase


Here we can see which invoices were an album purchase.

In [11]:
%%sql
SELECT Type_of_purchase,
COUNT(*) Ammount_of_invoice,
ROUND( CAST(COUNT(*) AS FLOAT)  /  CAST(
                                        (
                                        SELECT COUNT(invoice_id)
                                        FROM alb_inv_track
                                        ) 
                               AS FLOAT)
, 2)Percentage_of_invoice
FROM 
    (
    SELECT
     CASE
        WHEN
            (
                (
                SELECT inv_l.track_id FROM invoice_line inv_l
                WHERE inv_l.invoice_id = ait.invoice_id
            
                EXCEPT
                
                SELECT t.track_id FROM track t
                WHERE t.album_id = ait.album_id
                )
                
                    AND
                
                (
                SELECT t.track_id FROM track t
                WHERE t.album_id = ait.album_id
                EXCEPT
                
                SELECT inv_l.track_id FROM invoice_line inv_l
                WHERE inv_l.invoice_id = ait.invoice_id
                )    
            ) IS NULL THEN "Album Purchase"
        ELSE "Not Album Purchase"
    END Type_of_purchase
    FROM alb_inv_track ait
    )
GROUP BY Type_of_purchase;

Done.


Type_of_purchase,Ammount_of_invoice,Percentage_of_invoice
Album Purchase,167,0.27
Not Album Purchase,443,0.73


We updated the code to have the previous results as FROM. In this way we can perform actions like COUNT to help us get more detailed information like the percentage of invoices that were album purchases.

Here we can see that almost 3/4 of the customers do not seems to purchase music track per whole album. The shop should be better buying track individually, like for example, only the most famous tracks per album.